# First experiments with all LFI data

In [1]:
# Importing all packages
# normalizing flows packages
import torch
from torch.utils.data import TensorDataset, DataLoader
import normflows as nf

# loforest and locart functions
from CP2LFI.loforest import ConformalLoforest, tune_loforest_LFI
from CP2LFI.scores import Scores
from CP2LFI.simulations import Simulations, naive, predict_naive_quantile
from CP2LFI.posterior_models import normflow_posterior

# quantile regression
from sklearn.ensemble import HistGradientBoostingRegressor

# plotting and numpy
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm

# package to simulate from weinberg LFI data
# weinberg data (physics)
from hypothesis.benchmark import weinberg
# mg1 data (queue theory)
from hypothesis.benchmark import mg1
# multivariate normal (toy example)
from hypothesis.benchmark import tractable

## Organizing preliminary functions and classes first:

In [ ]:
class WaldoScore(Scores):
    def fit(self, X = None, thetas = None):
        # setting up model for normalizing flows
        if self.is_fitted:
            return self
        elif self.base_model is not None:
            self.base_model.fit(X, thetas)
        else:
            return self

    def compute(self, thetas, lambdas, B = 10**4):
        
        return lambdas

    def predict(self, thetas, cutoff):
        pred = np.vstack((thetas - cutoff, thetas + cutoff)).T
        return pred

## Starting by the multivariate normal data:

Simulating and testing for tractable:

In [2]:
simulator = tractable.Simulator()
prior = tractable.Prior()

# testing posterior model
# simulating from prior
inputs = prior.sample((10,)) # Draw 10 samples from the prior.
repeated_inputs = inputs.repeat(2, 1)

outputs = simulator(inputs)

Setting $n = 5$ and $B^{'} = 10000$ for posterior modeling:

In [3]:
B_model = 10000
n = 5
torch.manual_seed(45)
# simulating thetas
thetas = prior.sample((B_model, ))
repeated_thetas = thetas.repeat_interleave(repeats = n, dim = 0)

# simulating X's
X_sample = simulator(repeated_thetas)
X_dim = X_sample.shape[1]
X_sample

tensor([[-2.6983,  6.8049, -2.3008,  ..., -4.1367, -1.4454,  0.0833],
        [-4.2739, 12.4222, -2.8186,  ...,  5.3743, -3.5249, 10.2442],
        [-3.3973,  9.1984, -2.8430,  ..., -7.3831, -3.1952,  9.2909],
        ...,
        [-0.0912,  2.7111, -0.0203,  ...,  0.6607, -0.1690,  2.9145],
        [-1.1218,  4.3075, -0.2426,  ...,  3.0292, -0.0418,  2.8721],
        [-1.0765,  3.6011, -0.1130,  ...,  0.8354, -0.1374,  2.8256]])

In [4]:
X_dim = X_sample.shape[1]
X_net = X_sample.reshape(B_model, n * X_dim)

Fitting normalizing flow model:

In [5]:
nflow_post = normflow_posterior(
    latent_size = thetas.shape[1],
    sample_size = X_net.shape[1], 
    n_flows = 4, 
    hidden_units = 64, 
    hidden_layers = 2,
    enable_cuda = True,
    )

nflow_post.fit(X_net.numpy(), thetas.numpy(), patience = 50, n_epochs = 100)

AttributeError: 'normflow_posterior' object has no attribute 'context_size'

In [ ]:
nflow_post.plot_history()